In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import all required libraries

In [ ]:

# basic librareis
import zipfile
import glob
import os
import pandas as pd
import numpy as np
from PIL import Image
# plotting and visualizations
import matplotlib 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 
import missingno as msno
# preprocessing
from keras.preprocessing.image import (ImageDataGenerator, 
                                       img_to_array, 
                                       array_to_img, 
                                       load_img)

from sklearn.model_selection import train_test_split
# metrics
from sklearn.metrics import (confusion_matrix, 
                             classification_report, 
                             accuracy_score, 
                             f1_score, 
                             roc_auc_score)
# modeling
import tensorflow as tf
from keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten

from keras.applications import resnet50
from keras.applications.resnet50 import preprocess_input
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping,LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau



from keras import backend as K
K.clear_session()

# model plotting
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

# mesc
from sklearn.utils import shuffle

print('!Done')

In [ ]:
#extraction of train and test set from zipfiles to data folder
zip_files = glob.glob('/kaggle/input/dogs-vs-cats/*.zip')

print('{} files found in the input directory'.format(str(len(zip_files))) +'\n')
for file in zip_files:
    with zipfile.ZipFile(file, 'r') as Z:
        Z.extractall('data')
    print ('{} is extracted'.format(file.split('/')[-1]) + '\n')
      
print('Extraction is completed' + '\n')
    
  


In [ ]:
# Total number of images in train and test datasets
train_dir = '/kaggle/working/data/train/'
test_dir  = '/kaggle/working/data/test1/'
print('No. of Train Images:' + str(len(os.listdir(train_dir))))
print('No. of Test Images:' + str(len(os.listdir(test_dir))))


In [ ]:
# category and filepath extraction helper functions
def filename(path):
    return [file for file in os.listdir(path)]

def category(path):
    return [file.split('.')[0] for file in os.listdir(path)]

#image name and labes
x_train_imgname = filename(train_dir)
x_test_imgname = filename(test_dir)
y_train_label = category(train_dir)

#creation of total dataframe and submission dataframe
train_image_df = pd.DataFrame({ 'filename': x_train_imgname, 'category': y_train_label})
submission_image_df = pd.DataFrame({'filename': x_test_imgname})

print(train_image_df.head(7))
print('********************')
print(submission_image_df.head(7))



## Lets split data into training and validation dataset, and visualize the data distribution...

In [ ]:
# Data split into train data and validation data
train_valid_df, test_df = train_test_split(train_image_df, test_size = 0.04)
train_df, valid_df = train_test_split(train_valid_df, test_size = 0.2)
#train_valid_df = train_valid_df.reset_index(drop=True)
#test_df = test_df.reset_index(drop=True)


#train_images = train_valid_df.shape[0]
#valid_images = test_df.shape[0]
train_images = train_df.shape[0]
valid_images = valid_df.shape[0]
#holdon_images = test_df.shape[0]
#test_images = submission_image_df.shape[0]

print(train_valid_df.head(7))
print('********************')
print(test_df.head(7))
print('********************')
print(train_images)
print(valid_images)

In [ ]:
train_valid_df['category'].value_counts().plot.bar()

In [ ]:
test_df['category'].value_counts().plot.bar()

In [ ]:
train_df['category'].value_counts().plot.bar()

In [ ]:
valid_df['category'].value_counts().plot.bar()

## Decaying learing rate setting function

In [ ]:
epoch = 50
learning_rate = 3e-5 
lr_start = 0.00000001
lr_min = 0.000001
lr_max = 3e-5 
lr_rampup_epochs = 1
lr_sustain_epochs = 1
lr_exp_decay = .8

def lrfn(epoch):
    if epoch < lr_rampup_epochs:
        lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
    elif epoch < lr_rampup_epochs + lr_sustain_epochs:
        lr = lr_max
    else:
        lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
    return lr

# Classification with ResNet50

## Image agumentation data preparation with ImageDataGenerator

In [ ]:
size  = 224
batch_size = 128

In [ ]:
datagen = ImageDataGenerator(
                             rotation_range=10,
                             zoom_range=0.1,
                             horizontal_flip=True,
                             fill_mode='nearest',
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             preprocessing_function = preprocess_input)

X_train = datagen.flow_from_dataframe(
    train_df, 
    directory = train_dir, 
    x_col = 'filename',
    y_col = 'category',
    target_size= (size,size),
    class_mode = 'categorical',
    shuffle = True,
    batch_size = batch_size)


In [ ]:
datagenValidation = ImageDataGenerator(preprocessing_function = preprocess_input)

X_validation = datagenValidation.flow_from_dataframe(
    valid_df, 
    directory = train_dir, 
    x_col = 'filename',
    y_col = 'category',
    target_size= (size,size),
    class_mode = 'categorical',
    shuffle = True,
    batch_size = batch_size)

In [ ]:
test_df_1 = pd.DataFrame({ 'filename': test_df['filename']})
test_df_1.head(10)
#veo que con esto igual tengo el mismo resultado

In [ ]:
X_test = datagenValidation.flow_from_dataframe(
    test_df_1, 
    directory=train_dir, 
    x_col='filename',
    y_col=None, #Important
    class_mode=None,
    target_size= (size,size),
    shuffle=False
)

## Setup Callbacks

In [ ]:
earlystop = EarlyStopping(patience= 5)
    
lr_callback = LearningRateScheduler(lrfn, verbose = True)

callbacks = [earlystop, lr_callback]

## Building model for transfer learning on top of pretrained ResNet50 Model...

In [ ]:
#loading resent 
resNet = tf.keras.applications.ResNet50(weights = 'imagenet',
                        include_top = False,
                        input_shape = (224,224, 3))

resNet.trainable = False # Freeze layers
resNet_model = Sequential([
        resNet,
        Flatten(),
        Dense(1024, activation = 'relu'),
        Dropout(0.4),
        Dense(2, activation = 'softmax')])
     

optimizer = optimizers.Adam(1e-5)

resNet_model.summary()

## Lets train and validate Pre-trained ResNet50 model for top layers...

In [ ]:
resNet_model.compile(optimizer = optimizer,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])



resnet_history = resNet_model.fit(X_train, epochs = 15,
                          validation_data = X_validation,
                          validation_steps= valid_images//batch_size,
                          steps_per_epoch= train_images//batch_size,
                          callbacks = callbacks)


In [ ]:
acc = resnet_history.history['accuracy']
val_acc = resnet_history.history['val_accuracy']
loss = resnet_history.history['loss']
val_loss = resnet_history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Lets see ResNet predictions with confusion matrix ...

In [ ]:
res_pred = resNet_model.predict(X_test)

In [ ]:
res_pred

In [ ]:
import numpy as np
test_df['res_pred'] = np.argmax(res_pred, axis = -1)
labels = dict((v,k) for k,v in X_train.class_indices.items())
test_df['res_pred'] = test_df['res_pred'].map(labels)

In [ ]:
res_cf_matrix = confusion_matrix(test_df['category'],test_df['res_pred'])
res_cf_matrix

## Final predictions on Test Set

In [ ]:
res_sub_aug_map = ImageDataGenerator(preprocessing_function = preprocess_input)

res_sub_data = res_sub_aug_map.flow_from_dataframe(
             submission_image_df, test_dir,
             x_col = 'filename',
             y_col = None,
             class_mode = None,
             target_size = (size, size),
             shuffle = False)

In [ ]:
res_pred_sub = resNet_model.predict(res_sub_data)
submission_image_df['res_pred_sub'] = np.argmax(res_pred_sub, axis = -1)
labels = dict((v,k) for k,v in X_train.class_indices.items())
submission_image_df['res_pred_sub'] = submission_image_df['res_pred_sub'].map(labels)

print(submission_image_df.head())

In [ ]:
pred_sample = submission_image_df.sample(18)
pred_sample.reset_index(drop = True, inplace = True)

fig = plt.figure(figsize=(12,24))
fig.patch.set_facecolor('#f5f6f6')

for index, row in pred_sample.iterrows():
    filename = row['filename']
    res_pred = row['res_pred_sub']
    img = load_img( test_dir + filename, target_size= (size, size))
    plt.subplot(6,3, index+1)
    plt.imshow(img)
    plt.gca().axis('off')
    #plt.text(130, 175, 'vanila_pred: {}'.format(vani_pred), color='lightgreen',fontsize= 11, bbox=dict(facecolor='black', alpha=0.9))
    plt.text(130, 200, 'resNet_pred: {}'.format(res_pred), color='red',fontsize= 11, bbox=dict(facecolor='black', alpha=0.9))
    #plt.title(filename.split('.')[0])

plt.tight_layout()
#plt.subplots_adjust( wspace=0, hspace= 1)
fig.text(0,1, 'Hey Siri! is it cat or dog?: Test data labels',{'fontfamily':'serif','size':24,'weight':'bold'})
fig.show()
   